# Assignment 7

## Submit as an HTML file

### Print your name below

In [1]:
print("Meredith Teague")

Meredith Teague


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [2]:
# Write your answer here:
import pandas as pd 
import numpy as np 
import statsmodels.formula.api as smf




#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [5]:
data = pd.read_csv("data/results.csv")
print(data.head)

<bound method NDFrame.head of        resultId  raceId  driverId  constructorId number  grid position  \
0             1      18         1              1     22     1        1   
1             2      18         2              2      3     5        2   
2             3      18         3              3      7     7        3   
3             4      18         4              4      5    11        4   
4             5      18         5              1     23     3        5   
...         ...     ...       ...            ...    ...   ...      ...   
25835     25841    1096       854            210     47    12       16   
25836     25842    1096       825            210     20    16       17   
25837     25843    1096         1            131     44     5       18   
25838     25844    1096       849              3      6    20       19   
25839     25845    1096         4            214     14    10       \N   

      positionText  positionOrder  points  laps         time milliseconds  \
0   

### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [18]:
# Write your answer here
print(data.dtypes)  
subset = data.query("  milliseconds.str.isnumeric() == False"   )
#display(subset["milliseconds"])
list_old = ['\\N']
list_new = [np.nan]
data["milliseconds"] = data["milliseconds"].replace(list_old, list_new)

display(data["milliseconds"])




resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
race_time_ms       float64
dtype: object


0        5690616
1        5696094
2        5698779
3        5707797
4        5708630
          ...   
25835        NaN
25836        NaN
25837        NaN
25838        NaN
25839        NaN
Name: milliseconds, Length: 25840, dtype: object

### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [20]:
# Write your answer here

bins = [0, 3, 10, 20, float('inf')]  
labels = ['Podium', 'Points', 'Midfield', 'Backmarker']

data['finish_category'] = pd.cut(data['positionOrder'], bins=bins, labels=labels)
display(data['finish_category'])




0          Podium
1          Podium
2          Podium
3          Points
4          Points
           ...   
25835    Midfield
25836    Midfield
25837    Midfield
25838    Midfield
25839    Midfield
Name: finish_category, Length: 25840, dtype: category
Categories (4, object): ['Podium' < 'Points' < 'Midfield' < 'Backmarker']

### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [24]:
# Write your answer here
data['race_duration_minutes'] = data['milliseconds'].astype(float) / (1000 * 60)


avgraceduration = data.groupby('constructorId')['race_duration_minutes'].mean()

top5shortest = avgraceduration.nsmallest(5)
print(top5shortest)



constructorId
35    76.710777
29    77.604125
41    87.046767
16    89.428828
53    89.658852
Name: race_duration_minutes, dtype: float64


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [ ]:
# Write your answer here
driverStats = data.groupby('driverId').agg(
    avgfinishposition=('positionOrder', 'mean'),
    totalpoints=('points', 'sum'),
    racescompleted=('raceId', 'count'),
    bestfinish=('positionOrder', 'min')
)

driverstatssorted = driverStats.sort_values(by='total_points', ascending=False)

print(driverstatssorted.head(10))


          avg_finish_position  total_points  races_completed  best_finish
driverId                                                                 
1                    4.787097        4396.5              310            1
20                   7.093333        3098.0              300            1
4                    8.494413        2061.0              358            1
830                  6.533742        1983.5              163            1
8                    8.491477        1873.0              352            1
822                  7.601990        1778.0              201            1
3                    8.252427        1594.5              206            1
30                   6.879870        1566.0              308            1
817                  9.883621        1307.0              232            1
18                   9.695793        1235.0              309            1


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [27]:
# Write your answer here
data_clean = data[['points', 'grid', 'laps']].dropna()

model = smf.ols('points ~ grid + laps', data=data_clean).fit()

print(model.summary())

predicted_points = model.predict(pd.DataFrame({'grid': [3], 'laps': [55]}))
print(predicted_points)



                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Sun, 23 Mar 2025   Prob (F-statistic):               0.00
Time:                        09:48:33   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0